In [2]:
## import re
import numpy as np
import pandas as pd
import itertools
import cobra as c
import matplotlib.pyplot as plt
from CAFBAFY import *
from os import *

In [2]:
def Ratio_calc(E,P):
    np.random.seed(1)
    E.set_minimal_media()
    E.reactions.EX_glc__D_e.lower_bound=-1000.0
    E.optimize()
    P.set_minimal_media()
    sec = [x for x in E.reactions if x.x>0 and 'EX_' in x.id]
    for y in sec:
        try:
            if y.id == 'EX_ac_e':
                P.reactions.get_by_id(y.id).lower_bound = -y.x    
        except:
            continue
    P.optimize()
    RF_Ratio =  P.slim_optimize()/E.slim_optimize()
    return(RF_Ratio)


### CAFBAFY Enterobacter Models using default paramaters

In [3]:
df = pd.read_csv('../Data/Biggs_Model_list.csv')
E =  [x for x in df.Organism if 
    'Escherichia' in x or 
    'Salmonella'  in x or
    'Klebsiella'  in x or
    'Shigella'  in x or 
    'Yersinia'  in x
        ]
E_df = df[df['Organism'].isin(E)]
E_df = E_df.reset_index()

In [4]:
if not path.exists('../Data/Enterobacteriaceae'):
    makedirs('../Data/Enterobacteriaceae')
for mod1 in E_df.Models:
    try:
        if path.exists('../Data/Enterobacteriaceae/'  + mod1 + '.xml'):
            continue
        else:
            E = CAFBA_Model(c.io.read_sbml_model('../Data/Bigg_Model/'  + mod1 + '.xml'))
            if mod1 =='iECIAI1_1343':
                E.reactions.GLYOX_2.upper_bound=0.0 
            with E as m:
                m.set_minimal_media(ignore=['glc__D_e'])
                f = m.slim_optimize()
            if f<0.01:
                continue
                print(mod1)
                #IF the model does not grow on glucose no point in using (typically it's an auxitroph)
            else:
                E.quick_cafbafy()
                E.save('../Data/Enterobacteriaceae/'  + mod1 + '.xml')
    except:
        print(mod1)

Using license file /home/jean/gurobi811/gurobi.lic
Academic license - for non-commercial use only


### default_values + sensitivity analysis

In [5]:
E = CAFBA_Model(c.io.read_sbml_model('../Data/Enterobacteriaceae/iJO1366.xml'))
P = CAFBA_Model(c.io.read_sbml_model('../Data/Pseudomonadaceae/irrev_iJN1463.xml'))

Results are remarkably robust to the value of phi_q

In [6]:
def_phi_q  = E.reactions.cafba_rxn.upper_bound
high_phi_q = def_phi_q*2
low_phi_q = def_phi_q *0.5
E.reactions.cafba_rxn.upper_bound = high_phi_q
print('High phi_q of ' + str(high_phi_q)+ ' gives ratio of ' + str(Ratio_calc(E,P)))
E.reactions.cafba_rxn.upper_bound = low_phi_q
print('Low phi_q ' +  str(low_phi_q) +' gives ratio of '  + str(Ratio_calc(E,P)))
E.reactions.cafba_rxn.upper_bound = def_phi_q
print('Normal phi_q ' +  str(def_phi_q) +' gives ratio of '  + str(Ratio_calc(E,P)))

High phi_q of 0.968 gives ratio of 0.35035373914333173
Low phi_q 0.242 gives ratio of 0.369980260888942
Normal phi_q 0.484 gives ratio of 0.3567865649253747


Results are remarkably robust to the value of of wr

In [7]:
default_wr = E.reactions.BIOMASS_Ec_iJO1366_core_53p95M.metabolites[E.metabolites.CAFBA_met]
high_wr = default_wr*2
low_wr = default_wr*0.5
E.reactions.BIOMASS_Ec_iJO1366_core_53p95M.add_metabolites({E.metabolites.CAFBA_met: high_wr},combine=False)
print('High wr ' + str(high_wr)  +' gives ratio of '  + str(Ratio_calc(E,P)))
E.reactions.BIOMASS_Ec_iJO1366_core_53p95M.add_metabolites({E.metabolites.CAFBA_met: low_wr},combine=False)
print('Low wr ' + str(low_wr)  +' gives ratio of '  + str(Ratio_calc(E,P)))
E.reactions.BIOMASS_Ec_iJO1366_core_53p95M.add_metabolites({E.metabolites.CAFBA_met: default_wr},combine=False)
print('Normal wr ' + str(default_wr) +' gives ratio of '  + str(Ratio_calc(E,P)))

High wr -0.338 gives ratio of 0.36131645742417534
Low wr -0.0845 gives ratio of 0.3545216186759742
Normal wr -0.169 gives ratio of 0.3567865649253747


Results are remarkably robust to the value of of we

In [8]:
default_we  =-E.reactions.ATPS4rpp.metabolites[E.metabolites.CAFBA_met]
high_we = default_we * 2
low_we = default_we* 0.5
E.set_we(cost = high_we)
print('High we ' +  str(high_we) +' gives ratio of '  + str(Ratio_calc(E,P)))
E.set_we(cost = low_we)
print('Low we ' +  str(low_we) +' gives ratio of '  + str(Ratio_calc(E,P)))
E.set_we(cost = default_we)
print('Normal we ' + str(default_we)  +' gives ratio of '  + str(Ratio_calc(E,P)))

High we 0.001770325546512232 gives ratio of 0.3653733671926979
Low we 0.000442581386628058 gives ratio of 0.35259991372747834
Normal we 0.000885162773256116 gives ratio of 0.3567865649253747


Results are robust to small value of wc but high wc  causes the ratio to crash (because uptake rate growth becomes limited rather than proteome limited and E.coli switches to respiration rather than fermentaiton). In the paper we use wc = 0.0 because glucose is in excess

In [9]:
E.customCost(custom_costs={'EX_glc__D_e' : 0.0001})
print('wc = 0.0001 '  + str(Ratio_calc(E,P)))
E.customCost(custom_costs={'EX_glc__D_e' : 0.001})
print('wc = 0.001 '  + str(Ratio_calc(E,P)))
E.customCost(custom_costs={'EX_glc__D_e' : 0.01})
print('wc = 0.01 '  + str(Ratio_calc(E,P)))

wc = 0.0001 0.35682522633666275
wc = 0.001 0.2552669229965217
wc = 0.01 0.0109501536261946


Results are robust to applying a proteome allocation constraint on p.putida using the same approach as for E.coli

In [10]:
E.customCost(custom_costs={'EX_glc__D_e' : 0})
P.set_minimal_media(ignore=['glc__D_e'])
P.quick_cafbafy()
Ratio_calc(E,P)

w_e = 0.0008588967214616202


0.35678656492537675

This is even the case if growth on acetate is proteome limited

In [11]:
E.optimize()
P.set_minimal_media(ignore=['ac_e'])
P.reactions.EX_ac_e.lower_bound = -1000.0 #Acetate in excess
P.optimize()
print((P.reactions.BIOMASS_KT2440_WT3.x/-P.reactions.EX_ac_e.x)*(E.reactions.EX_ac_e.x/E.reactions.BIOMASS_Ec_iJO1366_core_53p95M.x))

0.35547356448226214


This is because Efficiency on acetate is only mildly affected by whether you are proteome or growth limited

In [12]:
P.set_minimal_media(ignore=['ac_e'])
#P.reactions.ATPM.lower_bound = 0.0
P.reactions.EX_ac_e.lower_bound = -1000.0
P.reactions.cafba_rxn.upper_bound = 0.484
P.optimize()
print((P.reactions.BIOMASS_KT2440_WT3.x/-P.reactions.EX_ac_e.x))
P.reactions.EX_ac_e.lower_bound = -10.0
P.reactions.cafba_rxn.upper_bound = 1000
P.optimize()
print((P.reactions.BIOMASS_KT2440_WT3.x/-P.reactions.EX_ac_e.x))


0.026894419752465558
0.026895946692224892


### Ratio for all pairs

In [13]:
# Perform CAFBA on all Enterobacter models to get biomass and acetate secretion
Ent_models = [f for f in listdir('../Data/Enterobacteriaceae/')]
sec = []
glc =[]
E_Biomass =[]
for i in Ent_models:
    print(i)
    E = CAFBA_Model(c.io.read_sbml_model('../Data/Enterobacteriaceae/'  + i))
    E.set_minimal_media(ignore=['glc__D_e'])
    t  = E.optimize()
    E_Biomass.append(t.objective_value)
    sec.append(dict(zip([x.id for x in E.reactions if 'EX_' in x.id and x.x>0],
         [x.x for x in E.reactions if 'EX_' in x.id and x.x>0])))
    glc.append(E.reactions.EX_glc__D_e.x)

iEcSMS35_1347.xml
iEcolC_1368.xml
iZ_1308.xml
iECED1_1282.xml
iEC1364_W.xml
iJO1366.xml
iLF82_1304.xml
e_coli_core.xml
iJR904.xml
iWFL_1372.xml
iEcHS_1320.xml
iECIAI39_1322.xml
iECO111_1330.xml
iB21_1397.xml
iEC1372_W3110.xml
iECIAI1_1343.xml
iAPECO1_1312.xml
iECABU_c1320.xml
iBWG_1329.xml
iE2348C_1286.xml
iAF1260.xml
iEC042_1314.xml
iYS1720.xml
iECOK1_1307.xml
STM_v1_0.xml
iETEC_1333.xml
iEC55989_1330.xml
iEC1368_DH5a.xml
iECB_1328.xml
iECD_1391.xml
iG2583_1286.xml
iY75_1357.xml
iECNA114_1301.xml
iECBD_1354.xml
iAF1260b.xml
iEcE24377_1341.xml
iECSE_1348.xml
iECSP_1301.xml
iECH74115_1262.xml
iECSF_1327.xml
iECDH1ME8569_1439.xml
iEC1344_C.xml
iUMNK88_1353.xml
iNRG857_1313.xml
iEKO11_1354.xml
iECP_1309.xml
iECs_1301.xml
iEC1349_Crooks.xml
iYL1228.xml
iML1515.xml
iUTI89_1310.xml
iECO103_1326.xml
iECO26_1355.xml
iUMN146_1321.xml
iSDY_1059.xml
iECW_1372.xml
iEcDH1_1363.xml
iECS88_1305.xml
iEC1356_Bl21DE3.xml


In [14]:
sec_df = pd.DataFrame(sec)
sec_df['EX_glc__D_e'] = glc
sec_df['Name'] = Ent_models
sec_df.to_csv('../Data/Raw/Enterobacter_Secretions.csv',index=False)

In [15]:
Pseud_Models = [f for f in listdir('../Data/Pseudomonadaceae/')]
for j in Pseud_Models:
    print(j)
    P = CAFBA_Model(c.io.read_sbml_model('../Data/Pseudomonadaceae/'  + j))
    P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
    P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
    ratio =[]
    ac=[]
    P_ac =[]
    E_glc =[]
    for k in range(0,len(Ent_models)):
        E_obj = E_Biomass[k]
        E_glc.append(E_obj)
        ac.append(sec[k]['EX_ac_e'])
        P.set_minimal_media()
        P.reactions.EX_ac_e.lower_bound =-sec[k]['EX_ac_e']
        P_ac.append(P.slim_optimize())
        P.reactions.EX_ac_e.lower_bound =0.0
        for p in sec[k].keys():
            try:
                P.reactions.get_by_id(p).lower_bound = -sec[k][p]
            except:
                continue
        P_obj = P.slim_optimize()
        if np.isnan(P_obj):
            P_obj = 0.0
        ratio.append(P_obj/E_obj)
    Temp_pairs_df = pd.DataFrame({'E': [x[:-4] for x in Ent_models],'P': j,'P/E':ratio,
                                     'glc':glc, 'ac' : ac,'E_Glucose' :E_glc,'P_acetate' :P_ac})
    if 'Pairs_df' in locals():
        Pairs_df = Pairs_df.append(Temp_pairs_df)
    else:
        Pairs_df = Temp_pairs_df


303.176_PID
303.429_PID
76869.5_PID
303.492_PID
1215087.7_PID
1042876.5_PID
303.175_PID
303.425_PID
303.212_PID
303.451_PID
303.410_PID
303.167_PID
303.452_PID
303.493_PID
303.171_PID
1150601.3_PID
303.424_PID
303.177_PID
303.447_PID
160488.4_PID
303.432_PID
1449986.3_PID
303.491_PID
1193146.3_PID
1410669.4_PID
303.532_PID
303.431_PID
390235.5_PID
303.533_PID
1193499.3_PID
931281.3_PID
303.473_PID
303.453_PID
303.529_PID
303.142_PID
303.189_PID
351746.6_PID
303.534_PID
1211579.3_PID
303.215_PID
303.486_PID
1410521.3_PID
303.397_PID
303.490_PID
303.209_PID
1331671.3_PID
303.489_PID
303.179_PID
1081940.5_PID
303.169_PID
303.178_PID
1215088.3_PID
303.531_PID
303.449_PID
303.199_PID
303.139_PID
303.535_PID
303.204_PID
303.214_PID
303.200_PID
303.180_PID
irrev_iJN1463.xml
303.430_PID
303.427_PID
1196325.3_PID
303.488_PID
303.411_PID
231023.4_PID
303.234_PID


### other pseudomonas metabolic models

Many of these models use non biggs notation which is a pain for however for all secretions the only ones they consume is acetate so i justset everything manually

In [16]:
#Pseudomonas aeruginosa PA01  easy no notation switch.
P = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iMO1056.xml'))
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
ratio =[]
ac=[]
P_ac =[]
E_glc =[]

for k in range(0,len(Ent_models)):
    E_obj = E_Biomass[k]
    E_glc.append(E_obj)
    P.set_minimal_media()
    ac.append(sec[k]['EX_ac_e'])
    P.reactions.EX_ac_e.lower_bound =-sec[k]['EX_ac_e']
    P_ac.append(P.slim_optimize())
    P.reactions.EX_ac_e.lower_bound =0.0
    for p in sec[k].keys():
        try:
            P.reactions.get_by_id(p).lower_bound = -sec[k][p]
        except:
            continue
    P_obj = P.slim_optimize()
    if np.isnan(P_obj):
        P_obj = 0.0
    ratio.append(P_obj/E_obj)
Temp_pairs_df = pd.DataFrame({'E': [x[:-4] for x in Ent_models],'P': 'iMO1056','P/E':ratio,
                                'glc':glc, 'ac' : ac,'E_Glucose' :E_glc,'P_acetate' :P_ac})
Pairs_df = Pairs_df.append(Temp_pairs_df)

In [17]:
%%capture
#Pseudomonas aeruginosa strain PA14 pain due to notation. does not contain methanol, glycolate or dhae so can ignore in secretions
P = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iPAU1129.xml'))
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
for x in P.reactions:
    if 'EX_' in x.id:
        x.lower_bound =0.0
#P.reactions.EX_cpd00029_LPAREN_e_RPAREN_.lower_bound=-0.0 #acetate
#P.reactions.EX_cpd00027_LPAREN_e_RPAREN_.lower_bound=-10.0 #glucose
P.reactions.EX_cpd00021_LPAREN_e_RPAREN_.lower_bound=-1000.0 #fe2
P.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound=-1000.0 #o2
P.reactions.EX_cpd00013_LPAREN_e_RPAREN_.lower_bound=-1000.0 #nh4
P.reactions.EX_cpd00067_LPAREN_e_RPAREN_.lower_bound=-1000.0 #H+
P.reactions.EX_cpd00009_LPAREN_e_RPAREN_.lower_bound=-1000.0 #phosphate
P.reactions.EX_cpd00048_LPAREN_e_RPAREN_.lower_bound=-1000.0 #sulfate
P.reactions.EX_cpd00011_LPAREN_e_RPAREN_.lower_bound=-1000.0 #c02
P.reactions.EX_cpd00254_LPAREN_e_RPAREN_.lower_bound=-1000.0 #magnesium
P.reactions.EX_cpd00971_LPAREN_e_RPAREN_.lower_bound=-1000.0 #sodium
P.reactions.EX_cpd00063_LPAREN_e_RPAREN_.lower_bound = -1000.0 #Calcium
P.reactions.EX_cpd00099_LPAREN_e_RPAREN_.lower_bound = -1000.0 #chlroine
P.reactions.EX_cpd00058_LPAREN_e_RPAREN_.lower_bound = -1000.0 #copper
P.reactions.EX_cpd00635_LPAREN_e_RPAREN_.lower_bound = -1000.0 #Cob(I)alamin
P.reactions.EX_cpd00001_LPAREN_e_RPAREN_.lower_bound = -1000.0 #H20
P.reactions.EX_cpd10516_LPAREN_e_RPAREN_.lower_bound = -1000.0 #fe3
P.reactions.EX_cpd00205_LPAREN_e_RPAREN_.lower_bound = -1000.0 #k+
P.reactions.EX_cpd00030_LPAREN_e_RPAREN_.lower_bound =-1000.0 #mn2
P.reactions.EX_cpd11574_LPAREN_e_RPAREN_.lower_bound =-1000.0# molybdate
P.reactions.EX_cpd00244_LPAREN_e_RPAREN_.lower_bound =-1000.0 #nickel
P.reactions.EX_cpd00034_LPAREN_e_RPAREN_.lower_bound =-1000.0 #Zinc
#tungstate absenct
#selenite absent
#Selinate absent
#cobalt absent
ratio =[]
ac=[]
P_ac =[]
E_glc =[]

for k in range(0,len(Ent_models)):
    E_obj = E_Biomass[k]
    E_glc.append(E_obj)
    ac.append(sec[k]['EX_ac_e'])
    P.reactions.get_by_id('EX_cpd00029_LPAREN_e_RPAREN_').lower_bound =-sec[k]['EX_ac_e']
    for p in sec[k].keys():
        try:
            if p == 'EX_ac_e':
                P.reactions.get_by_id('EX_cpd00029_LPAREN_e_RPAREN_').lower_bound = -sec[k][p]
        except:
            continue
    P_obj = P.slim_optimize()
    P_ac.append(P_obj)
    P.reactions.get_by_id('EX_cpd00029_LPAREN_e_RPAREN_').lower_bound = 0
    if np.isnan(P_obj):
        P_obj = 0.0
    ratio.append(P_obj/E_obj)
Temp_pairs_df = pd.DataFrame({'E': [x[:-4] for x in Ent_models],'P': 'iPAU1129','R/F':ratio,
                                'glc':glc, 'ac' : ac,'E_Glucose' :E_glc,'P_acetate' :P_ac})
Pairs_df = Pairs_df.append(Temp_pairs_df)

In [18]:
%%capture
#Pseudomonas aeruginosa strain PA01 pain due to notation. does not contain methanol, glycolate or dhae so can ignore in secretions
P = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iPAE1146.xml'))
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
for x in P.reactions:
    if 'EX_' in x.id:
        x.lower_bound =0.0
#P.reactions.EX_cpd00029_LPAREN_e_RPAREN_.lower_bound=-0.0 #acetate
#P.reactions.EX_cpd00027_LPAREN_e_RPAREN_.lower_bound=-10.0 #glucose
P.reactions.EX_cpd00021_LPAREN_e_RPAREN_.lower_bound=-1000.0 #fe2
P.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound=-1000.0 #o2
P.reactions.EX_cpd00013_LPAREN_e_RPAREN_.lower_bound=-1000.0 #nh4
P.reactions.EX_cpd00067_LPAREN_e_RPAREN_.lower_bound=-1000.0 #H+
P.reactions.EX_cpd00009_LPAREN_e_RPAREN_.lower_bound=-1000.0 #phosphate
P.reactions.EX_cpd00048_LPAREN_e_RPAREN_.lower_bound=-1000.0 #sulfate
P.reactions.EX_cpd00011_LPAREN_e_RPAREN_.lower_bound=-1000.0 #c02
P.reactions.EX_cpd00254_LPAREN_e_RPAREN_.lower_bound=-1000.0 #magnesium
P.reactions.EX_cpd00971_LPAREN_e_RPAREN_.lower_bound=-1000.0 #sodium
P.reactions.EX_cpd00063_LPAREN_e_RPAREN_.lower_bound = -1000.0 #Calcium
P.reactions.EX_cpd00099_LPAREN_e_RPAREN_.lower_bound = -1000.0 #chlroine
P.reactions.EX_cpd00058_LPAREN_e_RPAREN_.lower_bound = -1000.0 #copper
P.reactions.EX_cpd00635_LPAREN_e_RPAREN_.lower_bound = -1000.0 #Cob(I)alamin
P.reactions.EX_cpd00001_LPAREN_e_RPAREN_.lower_bound = -1000.0 #H20
P.reactions.EX_cpd10516_LPAREN_e_RPAREN_.lower_bound = -1000.0 #fe3
P.reactions.EX_cpd00205_LPAREN_e_RPAREN_.lower_bound = -1000.0 #k+
P.reactions.EX_cpd00030_LPAREN_e_RPAREN_.lower_bound =-1000.0 #mn2
P.reactions.EX_cpd11574_LPAREN_e_RPAREN_.lower_bound =-1000.0# molybdate
P.reactions.EX_cpd00244_LPAREN_e_RPAREN_.lower_bound =-1000.0 #nickel
P.reactions.EX_cpd00034_LPAREN_e_RPAREN_.lower_bound =-1000.0 #Zinc
#tungstate absenct
#selenite absent
#Selinate absent
#cobalt absent
ratio =[]
ac=[]
P_ac =[]
E_glc =[]

for k in range(0,len(Ent_models)):
    E_obj = E_Biomass[k]
    E_glc.append(E_obj)
    for p in sec[k].keys():
        try:
            if p == 'EX_ac_e':
                P.reactions.get_by_id('EX_cpd00029_LPAREN_e_RPAREN_').lower_bound = -sec[k][p]
                ac.append(sec[k][p])
        except:
            continue
    P_obj = P.slim_optimize()
    P_ac.append(P_obj)
    P.reactions.get_by_id('EX_cpd00029_LPAREN_e_RPAREN_').lower_bound = 0
    if np.isnan(P_obj):
        P_obj = 0.0
    ratio.append(P_obj/E_obj)
Temp_pairs_df = pd.DataFrame({'E': [x[:-4] for x in Ent_models],'P': 'iPAE1146','P/E':ratio,
                                'glc':glc, 'ac' : ac,'E_Glucose' :E_glc,'P_acetate' :P_ac})
Pairs_df = Pairs_df.append(Temp_pairs_df)

In [19]:
%%capture
#Pseudomonas stutzeri# 
#NB this is a massive pain as the ID are not bigg formated so i'm going to have to manually
#Map the secretions # I checked and only acetate has an exchagne reaction so that's fine.
P = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iPB890.xml'))
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
for x in P.reactions:
    if 'EX_' in x.id:
        x.lower_bound =0.0
P.reactions.EX_EC0011.lower_bound=-1000.0 #c02
P.reactions.EX_EC0021.lower_bound=-1000.0 #fe2
P.reactions.EX_EC0007.lower_bound=-1000.0 #o2
P.reactions.EX_EC0957.lower_bound=-1000.0 #nh4
P.reactions.EX_EC0065.lower_bound=-1000.0 #H+
P.reactions.EX_EC0009.lower_bound=-1000.0 #phosphate
P.reactions.EX_EC0048.lower_bound=-1000.0 #sulfate
P.reactions.EX_EC0248.lower_bound=-1000.0 #magnesium
P.reactions.EX_EC0954.lower_bound=-1000.0 #sodium
#Calcium absent
#Chlroine absent
P.reactions.EX_EC0056.lower_bound = -1000.0 #copper
#Cob(I)alamin absent
#H20 absent
##fe3 absent
P.reactions.EX_EC0197.lower_bound = -1000.0 #k+
P.reactions.EX_EC0030.lower_bound =-1000.0 #mn2
#molybdate absent
#nickel absent
P.reactions.EX_EC0034.lower_bound =-1000.0 #Zinc
#tungstate absenct
#selenite absent
#Selinate absent
P.reactions.EX_EC0144.lower_bound = -1000.0 #Cobalt
ratio =[]
ac=[]
P_ac =[]
E_glc =[]
for k in range(0,len(Ent_models)):
    E_obj = E_Biomass[k]
    E_glc.append(E_obj)
    for p in sec[k].keys():
        try:
            if p == 'EX_ac_e':
                P.reactions.get_by_id('EX_EC0029').lower_bound = -sec[k][p]
                ac.append(sec[k][p])
        except:
            continue
    P_obj = P.slim_optimize()
    P_ac.append(P_obj)
    P.reactions.get_by_id('EX_EC0029').lower_bound = 0
    if np.isnan(P_obj):
        P_obj = 0.0
    ratio.append(P_obj/E_obj)
Temp_pairs_df = pd.DataFrame({'E': [x[:-4] for x in Ent_models],'P': 'iPB890','P/E':ratio,
                                'glc':glc, 'ac' : ac,'E_Glucose' :E_glc,'P_acetate' :P_ac})
Pairs_df = Pairs_df.append(Temp_pairs_df)

In [20]:
%%capture
#Pseudomonas Flourescens# 
#this one flips the uptake reactions and has a weird naming convention so was a pain tomap. 
P = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iSB1139_COBRA.xml'))
P.reactions.get_by_id('BIOMASSRXN').lower_bound =0.0
P.reactions.get_by_id('BIOMASSRXN').upper_bound =1000.0
for x in P.reactions:
    if 'MIR' in x.id:
        x.lower_bound =-1000.0
        x.upper_bound =-0.0

    if x.id =='BIOMASSRXN':
        x.objective_coefficient =1
    else:
        x.objective_coefficient=0
P.reactions.MIRXN_32.upper_bound=1000.0 #fe2
P.reactions.MIRXN_77.upper_bound=1000.0 #o2
P.reactions.MIRXN_7.upper_bound=1000.0 #nh4
P.reactions.MIRXN_13.upper_bound =1000.0 #Cobalt
P.reactions.MIRXN_39.upper_bound=1000.0 #H+
P.reactions.MIRXN_79.upper_bound=1000.0 #phosphate
P.reactions.MIRXN_84.upper_bound=1000.0 #sulfate
P.reactions.MIRXN_12.upper_bound=1000.0 #c02
P.reactions.MIRXN_66.upper_bound=1000.0 #magnesium
P.reactions.MIRXN_73.upper_bound=1000.0 #sodium
P.reactions.MIRXN_93.upper_bound=1000.0 #Calcium
P.reactions.MIRXN_68.upper_bound=1000.0 #chlroine
#copper absent
P.reactions.MIRXN_11.upper_bound=1000.0 #Cob(I)alamin
P.reactions.MIRXN_41.upper_bound=1000.0 #H20
P.reactions.MIRXN_32.upper_bound=1000.0 #fe3
P.reactions.MIRXN_45.upper_bound=1000.0 #k+
P.reactions.MIRXN_71.upper_bound=1000.0 #mn2
# molyhbdenum absent
# nickel absent
P.reactions.MIRXN_96.upper_bound=1000.0 #Zinc
#tungstate absenct
#selenite absent
#Selinate absent
P.reactions.MIRXN_13.upper_bound = 1000.0 #Cobalt
ratio =[]
ac=[]
P_ac =[]
E_glc =[]

for k in range(0,len(Ent_models)):
    E_obj = E_Biomass[k]
    E_glc.append(E_obj)
    for p in sec[k].keys():
        try:
            if p == 'EX_ac_e':
                P.reactions.get_by_id('MIRXN_5').upper_bound = sec[k][p]
                ac.append(sec[k][p])
        except:
            continue
    P_obj = P.slim_optimize()
    P_ac.append(P_obj)
    P.reactions.get_by_id('MIRXN_5').upper_bound = 0
    if np.isnan(P_obj):
        P_obj = 0.0
    ratio.append(P_obj/E_obj)
Temp_pairs_df = pd.DataFrame({'E': [x[:-4] for x in Ent_models],'P': 'iSB1139','P/E':ratio,
                                'glc':glc, 'ac' : ac,'E_Glucose' :E_glc,'P_acetate' :P_ac})
Pairs_df = Pairs_df.append(Temp_pairs_df)

In [21]:
Pairs_df.to_csv('../Data/Raw/Pairs_Results.csv')

### Effect of Lactate Secretion on PE_Ratio

In the above analysis we have assumed that all of secreted metabolites produced by Enterobacter are accesible exclusively to the Pseudomonas. However we know that other organic acids are secreted in large amount, in particular many isolates secrete large amounts of lactate which is equally preferred by Enterobacteriacae and Psuedomonadacaea. 

We are going to model the effect of lactate secretion opn the predicted P/E ratio. To that end we will two approaches

In the 1st approach we assume the total secreted carbon is the same. E.coli grows acording to the original CAFBA solution and we magically transform a fraction of acetate to lactate (1.5 Acetate = 1 Lactate) . We will plot P/E ratio as a function of the fraction of secretion that is lactate

In the 2ndd approach (the one used for the paper) we use normal FBA and use empirically calculated Glucose, Acetate and Lactate levels to estimate the P/E Ratio.

In [22]:
## import re
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as pl
import cobra as c
from CAFBAFY import *
from os import *

In [23]:
#0.07 C-mole/L of gluocse = 0.011 mols of Glucose/L = 11.66 nmol/ul
#1 nmol/uL =   0.001 mols/litre
Empirical_Data = pd.read_csv('../Data/ES_48hGrowthAssayII_ALLMERGED_mono_20200113.csv')
Empirical_Data = Empirical_Data[Empirical_Data.ChromoType == 'B']
Empirical_Data = Empirical_Data[Empirical_Data.time_hours ==16]
E = CAFBA_Model(c.io.read_sbml_model('../Data/Bigg_Model/iJO1366.xml'))
E.set_minimal_media()
P = CAFBA_Model(c.io.read_sbml_model('../Data/Bigg_Model/iJN1463.xml'))
P.set_minimal_media()
Empirical_Data = Empirical_Data[Empirical_Data['CommunityReplicate'] != 'KAP']

In [24]:
PE_Ratio = []
Sanger_ID =[]
Condition =[]
for k in ['E_only','Both','P_only']:
    for j in set(Empirical_Data.SangerID):
        E.set_minimal_media()
        E.reactions.EX_glc__D_e.lower_bound = -11.66
        E.reactions.EX_ac_e.lower_bound = np.max(Empirical_Data[Empirical_Data.SangerID ==j].aceConc_nmol_uL)
        E.reactions.EX_succ_e.lower_bound = np.max(Empirical_Data[Empirical_Data.SangerID ==j].succConc_nmol_uL)
        E.reactions.EX_lac__D_e.lower_bound = np.max(Empirical_Data[Empirical_Data.SangerID ==j].lacConc_nmol_uL)
        f = E.slim_optimize()
        P.set_minimal_media()
        P.reactions.EX_ac_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].aceConc_nmol_uL)
        if k =='Both':
            P.reactions.EX_succ_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].succConc_nmol_uL)/2
            P.reactions.EX_lac__D_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].lacConc_nmol_uL)/2
        if k == 'P_only':
            P.reactions.EX_succ_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].succConc_nmol_uL)
            P.reactions.EX_lac__D_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].lacConc_nmol_uL)
        f2 = P.slim_optimize()
        
        E.set_minimal_media()
        if k =='Both':
            E.reactions.EX_succ_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].succConc_nmol_uL)/2
            E.reactions.EX_lac__D_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].lacConc_nmol_uL)/2
        if k == 'E_only':
            E.reactions.EX_succ_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].succConc_nmol_uL)
            E.reactions.EX_lac__D_e.lower_bound = -np.max(Empirical_Data[Empirical_Data.SangerID ==j].lacConc_nmol_uL)
        f3 = E.slim_optimize()
        
        if np.isnan(f3):
            f3 =0
        if np.isnan(f2):
            f2 =0
        PE_Ratio.append(f2/(f+f3))
        Sanger_ID.append(j)
        Condition.append(k)


In [25]:
df = pd.DataFrame({'Sanger_ID' : Sanger_ID,'Condition' : Condition, 'R_F' : PE_Ratio})
df.to_csv('../Data/Raw/Constrained_Secretions.csv',index=False)

### Oxygen_Limitation

In [26]:
Ent_models = [f for f in listdir('../Data/Enterobacteriaceae/')]
Pseud_models = [f for f in listdir('../Data/Pseudomonadaceae/')]
other_pseuds = ['iMO1056','iPAU1129','iPAE1146','iPB890','iSB1139_COBRA']
glc_an =[]
lac_an =[]
ac_an =[]
suc_an =[]
glc_aer =[]
lac_aer =[]
ac_aer =[]
suc_aer =[]

Every Enterobacteriacae

In [27]:
for i in Ent_models:
    X = CAFBA_Model(c.io.read_sbml_model('../Data/Enterobacteriaceae/'  + i))
    X.set_minimal_media()
    X.reactions.cafba_rxn.upper_bound = 1000 #Remove cafba constriant to do normal FBA
    
    X.reactions.EX_glc__D_e.lower_bound = -10.0
    X.reactions.EX_o2_e.lower_bound = -1000.0
    glc_aer.append(X.slim_optimize())
    X.reactions.EX_o2_e.lower_bound = -0.0
    glc_an.append(X.slim_optimize())
    X.reactions.EX_glc__D_e.lower_bound = -0.0

    X.reactions.EX_lac__D_e.lower_bound = -10.0
    X.reactions.EX_o2_e.lower_bound = -1000.0
    lac_aer.append(X.slim_optimize())
    X.reactions.EX_o2_e.lower_bound = -0.0
    lac_an.append(X.slim_optimize())
    X.reactions.EX_lac__D_e.lower_bound = -0.0

    X.reactions.EX_ac_e.lower_bound = -10.0
    X.reactions.EX_o2_e.lower_bound = -1000.0
    ac_aer.append(X.slim_optimize())
    X.reactions.EX_o2_e.lower_bound = -0.0
    ac_an.append(X.slim_optimize())
    X.reactions.EX_ac_e.lower_bound = -0.0

    X.reactions.EX_succ_e.lower_bound = -10.0
    X.reactions.EX_o2_e.lower_bound = -1000.0
    suc_aer.append(X.slim_optimize())
    X.reactions.EX_o2_e.lower_bound = -0.0
    suc_an.append(X.slim_optimize())
    X.reactions.EX_succ_e.lower_bound = -0.0


Every P.putida

In [28]:
for j in Pseud_models:
    X = CAFBA_Model(c.io.read_sbml_model('../Data/Pseudomonadaceae/'  + j))
    X.set_minimal_media()
    X.reactions.EX_glc__D_e.lower_bound = -10.0
    X.reactions.EX_o2_e.lower_bound = -1000.0
    glc_aer.append(X.slim_optimize())
    X.reactions.EX_o2_e.lower_bound = -0.0
    glc_an.append(X.slim_optimize())
    X.reactions.EX_glc__D_e.lower_bound = -0.0

    X.reactions.EX_lac__D_e.lower_bound = -10.0
    X.reactions.EX_o2_e.lower_bound = -1000.0
    lac_aer.append(X.slim_optimize())
    X.reactions.EX_o2_e.lower_bound = -0.0
    lac_an.append(X.slim_optimize())
    X.reactions.EX_lac__D_e.lower_bound = -0.0

    X.reactions.EX_ac_e.lower_bound = -10.0
    X.reactions.EX_o2_e.lower_bound = -1000.0
    ac_aer.append(X.slim_optimize())
    X.reactions.EX_o2_e.lower_bound = -0.0
    ac_an.append(X.slim_optimize())
    X.reactions.EX_ac_e.lower_bound = -0.0

    X.reactions.EX_succ_e.lower_bound = -10.0
    X.reactions.EX_o2_e.lower_bound = -1000.0
    suc_aer.append(X.slim_optimize())
    X.reactions.EX_o2_e.lower_bound = -0.0
    suc_an.append(X.slim_optimize())
    X.reactions.EX_succ_e.lower_bound = -0.0

Now for the horribly tedious task of modifying all the non-standard pesudomonas to measure these traits. Ugh

In [29]:
#Pseudomonas aeruginosa PA01  easy no notation switch.
P = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iMO1056.xml'))
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
P.reactions.get_by_id([x.id for x in P.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
P.set_minimal_media()
P.reactions.EX_glc_e.lower_bound = -10.0
P.reactions.EX_o2_e.lower_bound = -1000.0
glc_aer.append(P.slim_optimize())
P.reactions.EX_o2_e.lower_bound = -0.0
glc_an.append(P.slim_optimize())
P.reactions.EX_glc_e.lower_bound = -0.0

#no D lactate exchange in model
lac_aer.append(0)
lac_an.append(0)

P.reactions.EX_ac_e.lower_bound = -10.0
P.reactions.EX_o2_e.lower_bound = -1000.0
ac_aer.append(P.slim_optimize())
P.reactions.EX_o2_e.lower_bound = -0.0
ac_an.append(P.slim_optimize())
P.reactions.EX_ac_e.lower_bound = -0.0

P.reactions.EX_succ_e.lower_bound = -10.0
P.reactions.EX_o2_e.lower_bound = -1000.0
suc_aer.append(P.slim_optimize())
P.reactions.EX_o2_e.lower_bound = -0.0
suc_an.append(P.slim_optimize())
P.reactions.EX_succ_e.lower_bound = -0.0

In [30]:
%%capture
#Pseudomonas aeruginosa strain PA14 pain due to notation. does not contain methanol, glycolate or dhae so can ignore in secretions
P2 = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iPAU1129.xml'))
P2.reactions.get_by_id([x.id for x in P2.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
P2.reactions.get_by_id([x.id for x in P2.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
for x in P2.reactions:
    if 'EX_' in x.id:
        x.lower_bound =0.0

P2.reactions.EX_cpd00021_LPAREN_e_RPAREN_.lower_bound=-1000.0 #fe2
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound=-1000.0 #o2
P2.reactions.EX_cpd00013_LPAREN_e_RPAREN_.lower_bound=-1000.0 #nh4
P2.reactions.EX_cpd00067_LPAREN_e_RPAREN_.lower_bound=-1000.0 #H+
P2.reactions.EX_cpd00009_LPAREN_e_RPAREN_.lower_bound=-1000.0 #phosphate
P2.reactions.EX_cpd00048_LPAREN_e_RPAREN_.lower_bound=-1000.0 #sulfate
P2.reactions.EX_cpd00011_LPAREN_e_RPAREN_.lower_bound=-1000.0 #c02
P2.reactions.EX_cpd00254_LPAREN_e_RPAREN_.lower_bound=-1000.0 #magnesium
P2.reactions.EX_cpd00971_LPAREN_e_RPAREN_.lower_bound=-1000.0 #sodium
P2.reactions.EX_cpd00063_LPAREN_e_RPAREN_.lower_bound = -1000.0 #Calcium
P2.reactions.EX_cpd00099_LPAREN_e_RPAREN_.lower_bound = -1000.0 #chlroine
P2.reactions.EX_cpd00058_LPAREN_e_RPAREN_.lower_bound = -1000.0 #copper
P2.reactions.EX_cpd00635_LPAREN_e_RPAREN_.lower_bound = -1000.0 #Cob(I)alamin
P2.reactions.EX_cpd00001_LPAREN_e_RPAREN_.lower_bound = -1000.0 #H20
P2.reactions.EX_cpd10516_LPAREN_e_RPAREN_.lower_bound = -1000.0 #fe3
P2.reactions.EX_cpd00205_LPAREN_e_RPAREN_.lower_bound = -1000.0 #k+
P2.reactions.EX_cpd00030_LPAREN_e_RPAREN_.lower_bound =-1000.0 #mn2
P2.reactions.EX_cpd11574_LPAREN_e_RPAREN_.lower_bound =-1000.0# molybdate
P2.reactions.EX_cpd00244_LPAREN_e_RPAREN_.lower_bound =-1000.0 #nickel
P2.reactions.EX_cpd00034_LPAREN_e_RPAREN_.lower_bound =-1000.0 #Zinc

P2.reactions.EX_cpd00027_LPAREN_e_RPAREN_.lower_bound = -10.0
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -1000.0
glc_aer.append(P2.slim_optimize())
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -0.0
glc_an.append(P2.slim_optimize())
P2.reactions.EX_cpd00027_LPAREN_e_RPAREN_.lower_bound = -0.0

P2.reactions.EX_cpd00221_LPAREN_e_RPAREN_.lower_bound = -10.0
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -1000.0
lac_aer.append(P2.slim_optimize())
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -0.0
lac_an.append(P2.slim_optimize())
P2.reactions.EX_cpd00221_LPAREN_e_RPAREN_.lower_bound = -0.0

P2.reactions.EX_cpd00029_LPAREN_e_RPAREN_.lower_bound = -10.0
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -1000.0
ac_aer.append(P2.slim_optimize())
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -0.0
ac_an.append(P2.slim_optimize())
P2.reactions.EX_cpd00029_LPAREN_e_RPAREN_.lower_bound = -0.0

P2.reactions.EX_cpd00036_LPAREN_e_RPAREN_.lower_bound = -10.0
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -1000.0
suc_aer.append(P2.slim_optimize())
P2.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -0.0
suc_an.append(P2.slim_optimize())
P2.reactions.EX_cpd00036_LPAREN_e_RPAREN_.lower_bound = -0.0

In [31]:
%%capture
P3 = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iPAE1146.xml'))
P3.reactions.get_by_id([x.id for x in P3.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
P3.reactions.get_by_id([x.id for x in P3.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
for x in P3.reactions:
    if 'EX_' in x.id:
        x.lower_bound =0.0

P3.reactions.EX_cpd00021_LPAREN_e_RPAREN_.lower_bound=-1000.0 #fe2
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound=-1000.0 #o2
P3.reactions.EX_cpd00013_LPAREN_e_RPAREN_.lower_bound=-1000.0 #nh4
P3.reactions.EX_cpd00067_LPAREN_e_RPAREN_.lower_bound=-1000.0 #H+
P3.reactions.EX_cpd00009_LPAREN_e_RPAREN_.lower_bound=-1000.0 #phosphate
P3.reactions.EX_cpd00048_LPAREN_e_RPAREN_.lower_bound=-1000.0 #sulfate
P3.reactions.EX_cpd00011_LPAREN_e_RPAREN_.lower_bound=-1000.0 #c02
P3.reactions.EX_cpd00254_LPAREN_e_RPAREN_.lower_bound=-1000.0 #magnesium
P3.reactions.EX_cpd00971_LPAREN_e_RPAREN_.lower_bound=-1000.0 #sodium
P3.reactions.EX_cpd00063_LPAREN_e_RPAREN_.lower_bound = -1000.0 #Calcium
P3.reactions.EX_cpd00099_LPAREN_e_RPAREN_.lower_bound = -1000.0 #chlroine
P3.reactions.EX_cpd00058_LPAREN_e_RPAREN_.lower_bound = -1000.0 #copper
P3.reactions.EX_cpd00635_LPAREN_e_RPAREN_.lower_bound = -1000.0 #Cob(I)alamin
P3.reactions.EX_cpd00001_LPAREN_e_RPAREN_.lower_bound = -1000.0 #H20
P3.reactions.EX_cpd10516_LPAREN_e_RPAREN_.lower_bound = -1000.0 #fe3
P3.reactions.EX_cpd00205_LPAREN_e_RPAREN_.lower_bound = -1000.0 #k+
P3.reactions.EX_cpd00030_LPAREN_e_RPAREN_.lower_bound =-1000.0 #mn2
P3.reactions.EX_cpd11574_LPAREN_e_RPAREN_.lower_bound =-1000.0# molybdate
P3.reactions.EX_cpd00244_LPAREN_e_RPAREN_.lower_bound =-1000.0 #nickel
P3.reactions.EX_cpd00034_LPAREN_e_RPAREN_.lower_bound =-1000.0 #Zinc


P3.reactions.EX_cpd00027_LPAREN_e_RPAREN_.lower_bound = -10.0
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -1000.0
glc_aer.append(P3.slim_optimize())
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -0.0
glc_an.append(P3.slim_optimize())
P3.reactions.EX_cpd00027_LPAREN_e_RPAREN_.lower_bound = -0.0

P3.reactions.EX_cpd00221_LPAREN_e_RPAREN_.lower_bound = -10.0
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -1000.0
lac_aer.append(P3.slim_optimize())
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -0.0
lac_an.append(P3.slim_optimize())
P3.reactions.EX_cpd00221_LPAREN_e_RPAREN_.lower_bound = -0.0

P3.reactions.EX_cpd00029_LPAREN_e_RPAREN_.lower_bound = -10.0
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -1000.0
ac_aer.append(P3.slim_optimize())
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -0.0
ac_an.append(P3.slim_optimize())
P3.reactions.EX_cpd00029_LPAREN_e_RPAREN_.lower_bound = -0.0

P3.reactions.EX_cpd00036_LPAREN_e_RPAREN_.lower_bound = -10.0
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -1000.0
suc_aer.append(P3.slim_optimize())
P3.reactions.EX_cpd00007_LPAREN_e_RPAREN_.lower_bound = -0.0
suc_an.append(P3.slim_optimize())
P3.reactions.EX_cpd00036_LPAREN_e_RPAREN_.lower_bound = -0.0

In [32]:
%%capture
P4 = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iPB890.xml'))
P4.reactions.get_by_id([x.id for x in P4.reactions if x.objective_coefficient==1][0]).lower_bound =0.0
P4.reactions.get_by_id([x.id for x in P4.reactions if x.objective_coefficient==1][0]).upper_bound =1000.0
for x in P4.reactions:
    if 'EX_' in x.id:
        x.lower_bound =0.0
P4.reactions.EX_EC0011.lower_bound=-1000.0 #c02
P4.reactions.EX_EC0021.lower_bound=-1000.0 #fe2
P4.reactions.EX_EC0007.lower_bound=-1000.0 #o2
P4.reactions.EX_EC0957.lower_bound=-1000.0 #nh4
P4.reactions.EX_EC0065.lower_bound=-1000.0 #H+
P4.reactions.EX_EC0009.lower_bound=-1000.0 #phosphate
P4.reactions.EX_EC0048.lower_bound=-1000.0 #sulfate
P4.reactions.EX_EC0248.lower_bound=-1000.0 #magnesium
P4.reactions.EX_EC0954.lower_bound=-1000.0 #sodium
P4.reactions.EX_EC0056.lower_bound = -1000.0 #copper
P4.reactions.EX_EC0197.lower_bound = -1000.0 #k+
P4.reactions.EX_EC0030.lower_bound =-1000.0 #mn2
P4.reactions.EX_EC0034.lower_bound =-1000.0 #Zinc
P4.reactions.EX_EC0144.lower_bound = -1000.0 #Cobalt

P4.reactions.EX_EC0027.lower_bound = -10.0
P4.reactions.EX_EC0021.lower_bound = -1000.0
glc_aer.append(P4.slim_optimize())
P4.reactions.EX_EC0021.lower_bound = -0.0
glc_an.append(P4.slim_optimize())
P4.reactions.EX_EC0027.lower_bound = -0.0

#no D lactate exchange in model
lac_aer.append(0)
lac_an.append(0)

P4.reactions.EX_EC0029.lower_bound = -10.0
P4.reactions.EX_EC0021.lower_bound = -1000.0
ac_aer.append(P4.slim_optimize())
P4.reactions.EX_EC0021.lower_bound = -0.0
ac_an.append(P4.slim_optimize())
P4.reactions.EX_EC0029.lower_bound = -0.0

P4.reactions.EX_EC0036.lower_bound = -10.0
P4.reactions.EX_EC0021.lower_bound = -1000.0
suc_aer.append(P4.slim_optimize())
P4.reactions.EX_EC0021.lower_bound = -0.0
suc_an.append(P4.slim_optimize())
P4.reactions.EX_EC0036.lower_bound = -0.0

In [33]:
%%capture
P5 = CAFBA_Model(c.io.read_sbml_model('../Data/Published_Pseud_Models/iSB1139_COBRA.xml'))
P5.reactions.get_by_id('BIOMASSRXN').lower_bound =0.0
P5.reactions.get_by_id('BIOMASSRXN').upper_bound =1000.0
for x in P5.reactions:
    if 'MIR' in x.id:
        x.lower_bound =-1000.0
        x.upper_bound =-0.0

    if x.id =='BIOMASSRXN':
        x.objective_coefficient =1
    else:
        x.objective_coefficient=0
P5.reactions.MIRXN_32.upper_bound=1000.0 #fe2
P5.reactions.MIRXN_77.upper_bound=1000.0 #o2
P5.reactions.MIRXN_7.upper_bound=1000.0 #nh4
P5.reactions.MIRXN_13.upper_bound =1000.0 #Cobalt
P5.reactions.MIRXN_39.upper_bound=1000.0 #H+
P5.reactions.MIRXN_79.upper_bound=1000.0 #phosphate
P5.reactions.MIRXN_84.upper_bound=1000.0 #sulfate
P5.reactions.MIRXN_12.upper_bound=1000.0 #c02
P5.reactions.MIRXN_66.upper_bound=1000.0 #magnesium
P5.reactions.MIRXN_73.upper_bound=1000.0 #sodium
P5.reactions.MIRXN_93.upper_bound=1000.0 #Calcium
P5.reactions.MIRXN_68.upper_bound=1000.0 #chlroine
P5.reactions.MIRXN_11.upper_bound=1000.0 #Cob(I)alamin
P5.reactions.MIRXN_41.upper_bound=1000.0 #H20
P5.reactions.MIRXN_32.upper_bound=1000.0 #fe3
P5.reactions.MIRXN_45.upper_bound=1000.0 #k+
P5.reactions.MIRXN_71.upper_bound=1000.0 #mn2
P5.reactions.MIRXN_96.upper_bound=1000.0 #Zinc
P5.reactions.MIRXN_13.upper_bound = 1000.0 #Cobalt=

P5.reactions.MIRXN_22.upper_bound = 10.0
P5.reactions.MIRXN_77.upper_bound = 1000.0
glc_aer.append(P5.slim_optimize())
P5.reactions.MIRXN_77.upper_bound = 0.0
glc_an.append(P5.slim_optimize())
P5.reactions.MIRXN_22.upper_bound = 0.0

P5.reactions.MIRXN_24.upper_bound = 10.0
P5.reactions.MIRXN_77.upper_bound = 1000.0
lac_aer.append(P5.slim_optimize())
P5.reactions.MIRXN_77.upper_bound = 0.0
lac_an.append(P5.slim_optimize())
P5.reactions.MIRXN_24.upper_bound = 0.0

P5.reactions.MIRXN_5.upper_bound = 10.0
P5.reactions.MIRXN_77.upper_bound = 1000.0
ac_aer.append(P5.slim_optimize())
P5.reactions.MIRXN_77.upper_bound = 0.0
ac_an.append(P5.slim_optimize())
P5.reactions.MIRXN_5.upper_bound = 0.0

P5.reactions.MIRXN_83.upper_bound = 10.0
P5.reactions.MIRXN_77.upper_bound = 1000.0
suc_aer.append(P5.slim_optimize())
P5.reactions.MIRXN_77.upper_bound = 0.0
suc_an.append(P5.slim_optimize())
P5.reactions.MIRXN_83.upper_bound = 0.0

In [34]:
t = pd.DataFrame({'Model':  [x[:-4] for x in Ent_models] + Pseud_models + other_pseuds,
              'R_F' : ['F' for x in Ent_models] + ['R' for x in Pseud_models] + ['R' for x in other_pseuds],
              'Glucose_Aerobic':glc_aer,
              'Glucose_Anaerobic':glc_an,
              'Lactate_Aerobic':lac_aer,
              'Lactate_Anaerobic':lac_an,
              'Succinate_Aerobic':suc_aer,
              'Succinate_Anaerobic':suc_an,
              'Acetate_Aerobic':ac_aer,
              'Acetate_Anaerobic':ac_an})

In [35]:
t.to_csv('../Data/Raw/Oxygen_Limitation.csv')

### CAFBAFY Whole Genome Models Using Default Paramaters


In [36]:
## import re
import numpy as np
import pandas as pd
import itertools
import cobra as c
import matplotlib.pyplot as plt
from CAFBAFY import *
from os import *

In [37]:
%%capture
#Klebsiella
K = CAFBA_Model(c.io.read_sbml_model('../Data/Whole_Genome_Models/g6935_kbase.xml'))
C = CAFBA_Model(c.io.read_sbml_model('../Data/Whole_Genome_Models/g6938_kbase.xml'))
E = CAFBA_Model(c.io.read_sbml_model('../Data/Whole_Genome_Models/g6936_kbase.xml'))
R = CAFBA_Model(c.io.read_sbml_model('../Data/Whole_Genome_Models/g6937_kbase.xml'))
P = CAFBA_Model(c.io.read_sbml_model('../Data/Whole_Genome_Models/g6939_kbase.xml'))

In [38]:
K.quick_cafbafy(KBASE=True)
C.quick_cafbafy(KBASE=True)
E.quick_cafbafy(KBASE=True)
R.quick_cafbafy(KBASE=True)
P.reassign_compartments()

compartments are kbase messed up, manually reassigning compartments
w_e = 0.0009236807345769974
compartments are kbase messed up, manually reassigning compartments
w_e = 0.0009236913414289421
compartments are kbase messed up, manually reassigning compartments
w_e = 0.0009236807345769869
compartments are kbase messed up, manually reassigning compartments
w_e = 0.0009228141210215864
compartments are kbase messed up, manually reassigning compartments


In [39]:
def Ratio_calc_kbase(E,P):
    np.random.seed(1)
    E.set_minimal_media(KBASE=True)
    E.reactions.EX_cpd00027_e0.lower_bound=-1000.0        
    E.optimize()
    P.set_minimal_media(KBASE=True)
    sec = [x for x in E.reactions if x.x>0 and 'EX_' in x.id]
    for y in sec:
        try:
            if y.id == 'EX_cpd00029_e0':
                P.reactions.get_by_id(y.id).lower_bound = -y.x      
        except:
            continue
    P.optimize()
    RF_Ratio =  P.slim_optimize()/E.slim_optimize()
    df = pd.Series({'D_ace_glu' :-E.reactions.EX_cpd00029_e0.x/E.reactions.EX_cpd00027_e0.x,
             'w_ac_R' : - P.slim_optimize()/P.reactions.EX_cpd00029_e0.x,
             'w_glc_F' :-E.slim_optimize()/E.reactions.EX_cpd00027_e0.x,
             'R_F': P.slim_optimize()/E.slim_optimize()})
    return(df)


In [40]:
a = Ratio_calc_kbase(K,P)
b = Ratio_calc_kbase(C,P)
c = Ratio_calc_kbase(E,P)
d = Ratio_calc_kbase(R,P)
a['E']  = 'g6935'
a['E_Genus'] = 'Klebsiella'
b['E'] = 'g6938'
b['E_Genus'] = 'Citrobacter'
c['E'] = 'g6936'
c['E_Genus'] = 'Enterobacter'
d['E'] = 'g6939'
d['E_Genus'] = 'Raoultella'
fdf = pd.concat([a,b,c,d], axis=1).T
fdf['P'] = 'g6939'
fdf['P_Species'] = 'P.putida'
fdf.to_csv('../Data/Processed/Whole_Genome_R_F_Pred.csv',index=False)